In [1]:
#@title Aprendizaje Profundo | Otoño 2021 by Datitos{display-mode: "form" }
#@markdown ![71335171.png](data:image/png;base64,iVBORw0KGgoAAAANSUhEUgAAAMgAAADICAIAAAAiOjnJAAACwElEQVR4nOzdMY7iQBBA0WU197/FnJNNJ/FqWvLHZfd7McIGfVVQos3X+/3+A2f7e/UN8EzCIiEsEsIiISwSwiIhLBLCIiEsEsIiISwSwiIhLBLCIiEsEsIiISwSwiIhLBLCIiEsEsIiISwSwiIhLBLCIiEsEsIiISwSwiIhLBLCIiEsEsIiISwSwiIhLBLCIiEsEsIiISwSwiIhLBJfV9/A/7xer6XX3/1/gZ70eU0sEsIiISwSwiIhLBLCIiEsEiP2WEf7m9U9zVnvU9vh85pYJIRFQlgkhEVCWCSERUJYJEbssc5ytL+5at8zec9UM7FICIuEsEgIi4SwSAiLhLBIPGqPdWR1v1VfdwcmFglhkRAWCWGREBYJYZEQFokt9lhHdt4z1UwsEsIiISwSwiIhLBLCIiEsEh/dY+18zq4w7RzlTyYWCWGREBYJYZEQFglhkRAWiS1+jzVhr/Mbd7nP3zCxSAiLhLBICIuEsEgIi4SwSDxqj1U/72r1ulc9l2sCE4uEsEgIi4SwSAiLhLBICIvE7D3W9/fSy6/63dLqdZfvc/F7mMDEIiEsEsIiISwSwiIhLBLCIjF7j3WSs87rTXufyUwsEsIiISwSwiIhLBLCIiEsEq8Ju5N6r3OXc3xP2oeZWCSERUJYJIRFQlgkhEVCWCRG7LGOTN7TfNIdvwcTi4SwSAiLhLBICIuEsEgIi8Toc4U7Pyf9p8n7qiMmFglhkRAWCWGREBYJYZEQFonRe6wjd9zr7MbEIiEsEsIiISwSwiIhLBLCIiEsEsIiISwSwiIhLBLCIiEsEsIiISwSwiIhLBLCIiEsEsIiISwSwiIhLBLCIiEsEsIiISwSwiIhLBLCIiEsEsIiISwSwiIhLBLCIiEsEsIiISwSwiIhLBLCIiEsEsIiISwSwiIhLBLCIiEsEsIi8S8AAP//HtRtH09JwIEAAAAASUVORK5CYII=)
#El siguiente notebook fue traducido por Pablo Marinozi como el cuarto trabajo práctico correspondiente a la versión de Otoño del 2021 del curso Aprendizaje Profundo organizado por Datitos
#El trabajo original fue diseñado por los docentes de la CS231n: Convolutional Neural Networks for Visual Recognition de la Universidad de Stanford. http://cs231n.stanford.edu
#Para mayor información consultar https://datitos.github.io/curso-aprendizaje-profundo/#calendario

# Trabajo Práctico N°4: Redes Neuronales Convolucionales

# Tabla de contenido

Esta tarea tiene 4 partes. Vas a aprender PyTorch en **dos niveles diferentes de abstracción**, lo que te ayudará a comprenderlo mejor y a prepararte para el proyecto final.

1. **Parte I**: Preparación: utilizaremos el conjunto de datos CIFAR-10.
2. **Parte II**: API de Module de PyTorch: **Nivel de abstracción 1**, usaremos `nn.Module` para definir la arquitectura arbitraria de una red neuronal .
3. **Parte III**: API de Sequential de PyTorch: **Nivel de abstracción 2**, usaremos `nn.Sequential` para definir muy convenientemente una red de avance en serie .
4. **Parte IV**: Desafío abierto de CIFAR-10: implemente su propia red para obtener la mayor precisión posible en CIFAR-10. Puede experimentar con cualquier capa, optimizador, hiperparámetros u otras funciones avanzadas.

Aquí hay una tabla de comparación:

| API | Flexibilidad | Conveniencia |
| --------------- | ------------- | ------------- |
| `nn.Module` | Alta | Media |
| `nn.Sequential` | Baja | Alta | 

# Parte I.Preparación

Primero, cargamos el conjunto de datos CIFAR-10. Esto puede tardar un par de minutos la primera vez que lo hagás, pero los archivos deberían permanecer almacenados en caché después de eso.

PyTorch proporciona herramientas convenientes para automatizar el proceso de preprocesarlo e iterarlo en minibatches.

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F  # useful stateless functions
import torch.optim as optim
from torch.utils.data import DataLoader
from torch.utils.data import sampler

import torchvision.datasets as dset
import torchvision.transforms as T

import numpy as np

In [3]:
  !wget http://www.cs.toronto.edu/~kriz/cifar-10-python.tar.gz -O cifar-10-python.tar.gz
  !tar -xzvf cifar-10-python.tar.gz
  !rm cifar-10-python.tar.gz

--2021-05-02 15:48:26--  http://www.cs.toronto.edu/~kriz/cifar-10-python.tar.gz
Resolving www.cs.toronto.edu (www.cs.toronto.edu)... 128.100.3.30
Connecting to www.cs.toronto.edu (www.cs.toronto.edu)|128.100.3.30|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 170498071 (163M) [application/x-gzip]
Saving to: ‘cifar-10-python.tar.gz’

cifar-10-python.tar 100%[===================>] 162.60M  16.3MB/s    in 11s     

2021-05-02 15:48:38 (14.4 MB/s) - ‘cifar-10-python.tar.gz’ saved [170498071/170498071]

cifar-10-batches-py/
cifar-10-batches-py/data_batch_4
cifar-10-batches-py/readme.html
cifar-10-batches-py/test_batch
cifar-10-batches-py/data_batch_3
cifar-10-batches-py/batches.meta
cifar-10-batches-py/data_batch_2
cifar-10-batches-py/data_batch_5
cifar-10-batches-py/data_batch_1


In [4]:
NUM_TRAIN = 49000

# El paquete torchvision.transforms proporciona herramientas para preprocesar datos
# y para realizar el aumento de datos; aquí configuramos una variable transform para
# normalizar los datos restando el valor RGB medio y dividiendo por el
# desviación estándar de cada valor RGB; hemos "hardcodeado" la media y la desviación estándar.
transform = T.Compose([
                T.ToTensor(),
                T.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))
            ])

# Configuramos un objeto Dataset para cada split (train / val / test); torchvision.datasets carga
# ejemplos de entrenamiento uno a la vez, por lo que empaquetamos cada conjunto de datos en un DataLoader que
# itera a través del conjunto de datos y forma minibatches. Dividimos el conjunto de entrenamiento de CIFAR-10
# en los conjuntos train y val pasando un objeto Sampler al DataLoader indicando cómo debe muestrear el conjunto de datos subyacente.
cifar10_train = dset.CIFAR10('./cs231n/datasets', train=True, download=True,
                             transform=transform)
loader_train = DataLoader(cifar10_train, batch_size=64, 
                          sampler=sampler.SubsetRandomSampler(range(NUM_TRAIN)))

cifar10_val = dset.CIFAR10('./cs231n/datasets', train=True, download=True,
                           transform=transform)
loader_val = DataLoader(cifar10_val, batch_size=64, 
                        sampler=sampler.SubsetRandomSampler(range(NUM_TRAIN, 50000)))

cifar10_test = dset.CIFAR10('./cs231n/datasets', train=False, download=True, 
                            transform=transform)
loader_test = DataLoader(cifar10_test, batch_size=64)


Extracting ./cs231n/datasets/cifar-10-python.tar.gz to ./cs231n/datasets
Files already downloaded and verified
Files already downloaded and verified


Tenés la opción de **usar GPU configurando la variable `USE_GPU` en True en la celda de abajo**. No es estrictamente necesario utilizar GPU para este práctico, aunque habilitarla ocasionará que el entrenamiento demore menos. Tené en cuenta que si tu computadora no tiene CUDA habilitado, `torch.cuda.is_available ()` devolverá False y este notebook regresará al modo CPU.

Las variables globales `dtype` y` device` controlarán los tipos de datos a lo largo de este práctico.

## Usuarios de Colab

Si estás utilizando Colab, tenés que cambiar manualmente a un dispositivo GPU. Podés hacer esto haciendo clic en `Entorno de ejecución -> Cambiar tipo de entorno de ejecución` y seleccionando` GPU` en `Acelerador de hardware`. Tené en cuenta que tenés que volver a ejecutar las celdas desde arriba, porque el kernel se reinicia al cambiar los entornos de ejecución.

In [5]:
USE_GPU = True

dtype = torch.float32 # vamos a usar float a lo largo de este tutorial

if USE_GPU and torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')

# Constante para controlar la frecuencia con la que imprimimos la pérdida durante el entrenamiento
print_every = 100

print('using device:', device)

using device: cuda


# Parte III. API Module de PyTorch

PyTorch proporciona la API `nn.Module` para que definas arquitecturas de red arbitrarias, mientras realizás un seguimiento de todos los parámetros aprendibles. PyTorch también proporciona el paquete `torch.optim` que implementa todos los optimizadores comunes, como RMSProp, Adagrad y Adam.

Para utilizar la API Module, siga los pasos a continuación:

1. Genere una subclase de `nn.Module`. Dé a su clase de red un nombre intuitivo como "2CapasMLP".

2. En el constructor `__init __ ()`, defina todas las capas que necesita como atributos de clase. Los objetos de capa como `nn.Linear` y` nn.Conv2d` son en sí mismos subclases de `nn.Module` y contienen parámetros que se pueden aprender, por lo que no tenés que crear una instancia para esos tensores. `nn.Module` monitoreará estos parámetros internos por vos. Consultá la [documentación] (http://pytorch.org/docs/master/nn.html) para obtener más información sobre las docenas de capas integradas en Pytorch. ** Advertencia **: ¡no te olvidés dellamar al `super () .__ init __ ()` primero!**

3. En el método `forward()`, definí la *conectividad* de tu red. Tenés usar los atributos definidos en `__init__` como llamadas de función que toman el tensor como entrada y generan como salida al tensor "transformado". **No** creés capas nuevas con parámetros que se puedan aprender en `forward()`! Todos ellos tienen que declararse por adelantado en `__init__`.

Después de definir la subclase de Module, podés instanciarla como un objeto y llamarla como una función.

### API Module: MLP de dos capas
A continuación, se muestra un ejemplo concreto de un MLP de dos capas

In [6]:
def flatten(x):
    N = x.shape[0] # lee un tensor con dimensiones N, C, H, W
    return x.view(N, -1)  # "aplana" las componentes C * H * W ven un solo vector por imagen

def test_flatten():
    x = torch.arange(12).view(2, 1, 3, 2)
    print('Antes de flatten: ', x,'\n')
    print('Después de flatten: ', flatten(x))

test_flatten()

Antes de flatten:  tensor([[[[ 0,  1],
          [ 2,  3],
          [ 4,  5]]],


        [[[ 6,  7],
          [ 8,  9],
          [10, 11]]]]) 

Después de flatten:  tensor([[ 0,  1,  2,  3,  4,  5],
        [ 6,  7,  8,  9, 10, 11]])


In [7]:
class DosCapasMLP(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super().__init__()
        # asigna instancias de capas a los atributos de la clase
        self.fc1 = nn.Linear(input_size, hidden_size)
        # nn.init contiene métodos de inicialización convenientes
        # http://pytorch.org/docs/master/nn.html#torch-nn-init 
        nn.init.kaiming_normal_(self.fc1.weight)
        self.fc2 = nn.Linear(hidden_size, num_classes)
        nn.init.kaiming_normal_(self.fc2.weight)
    
    def forward(self, x):
        # forward siempre define la conectividad
        x = flatten(x)
        scores = self.fc2(F.relu(self.fc1(x)))
        return scores

def test_DosCapasMLP():
    input_size = 50
    x = torch.zeros((64, input_size), dtype=dtype)  # minibatch size 64, feature dimension 50
    model = DosCapasMLP(input_size, 42, 10)
    scores = model(x)
    print(scores.size())  # deberías ver [64, 10]
test_DosCapasMLP()

torch.Size([64, 10])


###API Module: CNN de 3 Capas
Es tu turno de implementar una CNN de 3 capas seguida de una capa densa. La arquitectura de la red debe ser la siguiente:

1. Capa convolucional con una cantidad `channel_1` de filtros de 5x5 con padding=2
2. ReLU
3. Capa convolucional con una cantidad `channel_2` de filtros 3x3 con padding=1
4. ReLU
5. Capa densa con una cantidad `num_classes` de neuronas de salida.

Tenés que inicializar las matrices de peso del modelo utilizando el método de inicialización normal de Kaiming.

**PISTA**: http://pytorch.org/docs/stable/nn.html#conv2d

Después de implementar la CNN de tres capas, la función `test_TresCapasCNN` ejecutará su implementación; debería imprimir `(64, 10)`.

In [8]:
class TresCapasCNN(nn.Module):
    def __init__(self, in_channel, channel_1, channel_2, num_classes):
        super().__init__()
        ########################################################################
        # TO_DO: Configure las capas que necesita para la CNN de tres capas con#
        # la arquitectura definida anteriormente.                              #
        ########################################################################
        # ***** INICIO DE TU CÓDIGO (NO BORRES NI MODIFIQUES ESTA LÍNEA) ***** 

        self.capa1 = nn.Conv2d(in_channel, channel_1, kernel_size=5, padding=2)
        nn.init.kaiming_normal_(self.capa1.weight)
        self.capa2 =nn.Conv2d(channel_1, channel_2, kernel_size=3, padding=1)
        nn.init.kaiming_normal_(self.capa2.weight)
        self.flatten =  nn.Flatten()
        self.salida = nn.Linear(32*32*channel_2, 10)
        nn.init.kaiming_normal_(self.salida.weight)

        # ***** FINAL DE TU CÓDIGO (NO BORRES NI MODIFIQUES ESTA LÍNEA) *****
        ########################################################################
        #                          FINAL DE TU CÓDIGO                          #       
        ########################################################################

    def forward(self, x):
        scores = None
        ########################################################################
        # TODO: Implementá la función forward para la CNN de 3 capas. Deberías #
        # usar las capas que definiste en __init__ y especificar la            #
        # conectividad de dichas capas                                         #
        ########################################################################
        # ***** INICIO DE TU CÓDIGO (NO BORRES NI MODIFIQUES ESTA LÍNEA) ***** 

        scores = F.relu(self.capa2(F.relu(self.capa1(x))))
        scores = self.salida(self.flatten(scores))

        # ***** FINAL DE TU CÓDIGO (NO BORRES NI MODIFIQUES ESTA LÍNEA) *****
        ########################################################################
        #                          FINAL DE TU CÓDIGO                          #       
        ########################################################################
        return scores


def test_TresCapasCNN():
    x = torch.zeros((64, 3, 32, 32), dtype=dtype)  # minibatch size 64, image size [3, 32, 32]
    model = TresCapasCNN(in_channel=3, channel_1=12, channel_2=8, num_classes=10)
    scores = model(x)
    print(scores.size())  # deberías ver [64, 10]
test_TresCapasCNN()

torch.Size([64, 10])


### Module API: Calcular Accuracy
Dado el conjunto de pruebas o de validación, podemos verificar el accuracy de clasificación de una red neuronal.


In [9]:
def check_accuracy_part34(loader, model):
    if loader.dataset.train:
        print('Checking accuracy on validation set')
    else:
        print('Checking accuracy on test set')   
    num_correct = 0
    num_samples = 0
    model.eval()  # set model to evaluation mode
    with torch.no_grad():
        for x, y in loader:
            x = x.to(device=device, dtype=dtype)  # move to device, e.g. GPU
            y = y.to(device=device, dtype=torch.long)
            scores = model(x)
            _, preds = scores.max(1)
            num_correct += (preds == y).sum()
            num_samples += preds.size(0)
        acc = float(num_correct) / num_samples
        print('Got %d / %d correct (%.2f)' % (num_correct, num_samples, 100 * acc))

### Module API: Bucle de entrenamiento
En lugar de actualizar los valores de los pesos nosotros mismos, utilizamos un objeto Optimizer del paquete `torch.optim`, que abstrae la noción de un algoritmo de optimización y proporciona implementaciones de la mayoría de los algoritmos comúnmente utilizados para esta tarea.

In [10]:
def train_part34(model, optimizer, epochs=1):
    """
    Entrena un modelo en CIFAR-10 usando la API Module de PyTorch.

    Entradas:
    - model: Un Module de PyTorch que da el modelo a entrenar.
    - optimizer: un objeto Optimizador que usaremos para entrenar el modelo
    - epochs: (Opcional) Un número entero de Python que da el número de épocas para entrenar

    Devuelve: nada, pero imprime la precisión del modelo durante el entrenamiento.
    """ 
    model = model.to(device=device)  # mueve los parámetros del modelo to CPU/GPU
    for e in range(epochs):
        print('Epoch %d' % (e))
        for t, (x, y) in enumerate(loader_train):
            model.train()  # pone el modelo en modo entrenamiento
            x = x.to(device=device, dtype=dtype)  # mover a device, e.g. GPU
            y = y.to(device=device, dtype=torch.long)

            scores = model(x)
            loss = F.cross_entropy(scores, y)

            # Pone a cero todos los gradientes de las variables que el optimizador
            # actualizará.
            optimizer.zero_grad()

            # Este es el paso de propagación hacia atrás: calcula el gradiente 
            # de la función de pérdida con respecto a cada parámetro del modelo.
            loss.backward()

            # Efectivamente actualiza los parámetros del modelo usando los gradientes
            # calculado por el paso anterior
            optimizer.step()

            if t % print_every == 0:
                print('Iteration %d, loss = %.4f' % (t, loss.item()))
                check_accuracy_part34(loader_val, model)
                print()

### API Module: Entrenamiento de un MLP de dos capas
Ahora estamos listos para ejecutar el ciclo de entrenamiento. 

Simplemente pasá el tamaño de la entrada, el tamaño de la capa oculta y el número de clases (es decir, el tamaño de salida) al constructor de `DosCapasMLP`.

También necesitas definir un optimizador que monitoree todos los parámetros que se pueden aprender dentro de `DosCapasMLP`.

No tenés que ajustar ningún hiperparámetro, pero deberías obtener un accuracy del modelo por encima del 40% después de una sola época de entrenamiento .

In [11]:
hidden_layer_size = 4000
learning_rate = 1e-2
model = DosCapasMLP(3 * 32 * 32, hidden_layer_size, 10)
optimizer = optim.SGD(model.parameters(), lr=learning_rate)

train_part34(model, optimizer)

Epoch 0
Iteration 0, loss = 3.7353
Checking accuracy on validation set
Got 161 / 1000 correct (16.10)

Iteration 100, loss = 2.5313
Checking accuracy on validation set
Got 312 / 1000 correct (31.20)

Iteration 200, loss = 1.9454
Checking accuracy on validation set
Got 401 / 1000 correct (40.10)

Iteration 300, loss = 1.8251
Checking accuracy on validation set
Got 377 / 1000 correct (37.70)

Iteration 400, loss = 1.7568
Checking accuracy on validation set
Got 383 / 1000 correct (38.30)

Iteration 500, loss = 1.9504
Checking accuracy on validation set
Got 417 / 1000 correct (41.70)

Iteration 600, loss = 1.7776
Checking accuracy on validation set
Got 438 / 1000 correct (43.80)

Iteration 700, loss = 1.8119
Checking accuracy on validation set
Got 429 / 1000 correct (42.90)



###API Module: Entrenamiento de una CNN de 3 Capas
Ahora deberías usar la API de Module para entrenar una CNN de tres capas en CIFAR. ¡Esto debería ser muy similar al entrenamiento de la red de dos capas! No necesitás ajustar ningún hiperparámetro, pero deberías alcanzar más del 45% después de entrenar durante una época.

Deberías entrenar el modelo usando un descenso de gradiente estocástico sin momentum.

In [12]:
learning_rate = 3e-3
channel_1 = 32
channel_2 = 16

model = None
optimizer = None
#############################################################################
# TODO: Creá una instancia de TresCapasCNN y un optimizador correspondiente #
#############################################################################
# ***** INICIO DE TU CÓDIGO (NO BORRES NI MODIFIQUES ESTA LÍNEA) ***** 

model = TresCapasCNN(3,channel_1, channel_2, 10)
optimizer = optim.SGD(model.parameters(), lr=learning_rate, momentum = 0)

# ***** FINAL DE TU CÓDIGO (NO BORRES NI MODIFIQUES ESTA LÍNEA) *****       #
#############################################################################
#                          FINAL DE TU CÓDIGO                               #       
#############################################################################

train_part34(model, optimizer)

Epoch 0
Iteration 0, loss = 2.9369
Checking accuracy on validation set
Got 122 / 1000 correct (12.20)

Iteration 100, loss = 1.7666
Checking accuracy on validation set
Got 315 / 1000 correct (31.50)

Iteration 200, loss = 1.9538
Checking accuracy on validation set
Got 391 / 1000 correct (39.10)

Iteration 300, loss = 1.9326
Checking accuracy on validation set
Got 380 / 1000 correct (38.00)

Iteration 400, loss = 1.3583
Checking accuracy on validation set
Got 442 / 1000 correct (44.20)

Iteration 500, loss = 1.6253
Checking accuracy on validation set
Got 446 / 1000 correct (44.60)

Iteration 600, loss = 1.3650
Checking accuracy on validation set
Got 474 / 1000 correct (47.40)

Iteration 700, loss = 1.3943
Checking accuracy on validation set
Got 482 / 1000 correct (48.20)



# Part III. API Sequential de Pytorch

La Parte II presentó la API Module de PyTorch, que le permite definir capas de aprendizaje arbitrarias y su conectividad.

Para modelos simples como una serie de capas apiladas, aún necesita seguir 3 pasos: generar una subclase de `nn.Module`, asignar capas a atributos de clase en` __init__`, y llamar a cada capa una por una en `forward ()`. ¿Existe una forma más conveniente?

Afortunadamente, PyTorch proporciona un contenedor subclase de Module llamado `nn.Sequential`, que fusiona los pasos anteriores en uno. No es tan flexible como `nn.Module`, porque no puede especificar una topología más compleja que una serie de capas apiladas, pero es lo suficientemente bueno para muchos casos de uso.

### API Sequential: MLP de Dos Capas
Veamos cómo reescribir nuestro ejemplo de MLP de dos capas con `nn.Sequential`, y entrenarlo usando el ciclo de entrenamiento definido anteriormente.

Nuevamente, no tenés que ajustar ningún hiperparámetro acá, pero deberías lograr una precisión superior al 40% después de una época de entrenamiento. 

In [13]:
# Necesitamos envolver la función `flatten` en un Module para apilarlo
# en nn.Sequential
class Flatten(nn.Module):
    def forward(self, x):
        return flatten(x)

hidden_layer_size = 4000
learning_rate = 1e-2

model = nn.Sequential(
    Flatten(),
    nn.Linear(3 * 32 * 32, hidden_layer_size),
    nn.ReLU(),
    nn.Linear(hidden_layer_size, 10),
)

# podés utilizar el momentum de Nesterov en optim.SGD
optimizer = optim.SGD(model.parameters(), lr=learning_rate,
                     momentum=0.9, nesterov=True)

train_part34(model, optimizer)

Epoch 0
Iteration 0, loss = 2.3586
Checking accuracy on validation set
Got 156 / 1000 correct (15.60)

Iteration 100, loss = 2.0532
Checking accuracy on validation set
Got 386 / 1000 correct (38.60)

Iteration 200, loss = 1.8986
Checking accuracy on validation set
Got 399 / 1000 correct (39.90)

Iteration 300, loss = 1.7846
Checking accuracy on validation set
Got 416 / 1000 correct (41.60)

Iteration 400, loss = 1.9209
Checking accuracy on validation set
Got 409 / 1000 correct (40.90)

Iteration 500, loss = 1.8483
Checking accuracy on validation set
Got 433 / 1000 correct (43.30)

Iteration 600, loss = 1.7829
Checking accuracy on validation set
Got 417 / 1000 correct (41.70)

Iteration 700, loss = 1.6395
Checking accuracy on validation set
Got 448 / 1000 correct (44.80)



### API Sequential: CNN de 3 capas
Acá tenés que usar `nn.Sequential` para definir y entrenar una CNN de tres capas con la misma arquitectura que usamos en la Parte II:

1. Capa convolucional (con sesgo) con 32 filtros de 5x5, con padding de 2
2. ReLU
3. Capa convolucional (con sesgo) con 16 filtros 3x3, con padding de 1
4. ReLU
5. Capa densa (con sesgo) para calcular las puntuaciones de 10 clases

Tenés que optimizar tu modelo usando el descenso de gradiente estocástico con el momentum de Nesterov en 0.9.

De nuevo, no necesitás ajustar ningún hiperparámetro, pero deberías ver un accuracy superior al 55% después de una época de entrenamiento. 

In [14]:
channel_1 = 32
channel_2 = 16
learning_rate = 1e-2

model = None
optimizer = None

#############################################################################
# TODO: Reescribí la CNN de 3 capas con sesgo de la Parte II con la         #
# API Sequential.                                                           #
#############################################################################
# ***** INICIO DE TU CÓDIGO (NO BORRES NI MODIFIQUES ESTA LÍNEA) ***** 

in_c = 3
out = 10

model = nn.Sequential(
    nn.Conv2d(in_c, channel_1, kernel_size=5, padding=2), 
    nn.ReLU(),
    nn.Conv2d(channel_1, channel_2, kernel_size=3, padding=1), 
    nn.ReLU(),
    Flatten(),
    nn.Linear(32*32*channel_2, out),
)


optimizer = optim.SGD(model.parameters(), lr=learning_rate,
                     momentum=0.9, nesterov=True)


# ***** FINAL DE TU CÓDIGO (NO BORRES NI MODIFIQUES ESTA LÍNEA) *****       #
#############################################################################
#                          FINAL DE TU CÓDIGO                               #       
#############################################################################

train_part34(model, optimizer)

Epoch 0
Iteration 0, loss = 2.3086
Checking accuracy on validation set
Got 96 / 1000 correct (9.60)

Iteration 100, loss = 1.4128
Checking accuracy on validation set
Got 405 / 1000 correct (40.50)

Iteration 200, loss = 1.4094
Checking accuracy on validation set
Got 489 / 1000 correct (48.90)

Iteration 300, loss = 1.7440
Checking accuracy on validation set
Got 507 / 1000 correct (50.70)

Iteration 400, loss = 1.1580
Checking accuracy on validation set
Got 522 / 1000 correct (52.20)

Iteration 500, loss = 1.1095
Checking accuracy on validation set
Got 566 / 1000 correct (56.60)

Iteration 600, loss = 1.2839
Checking accuracy on validation set
Got 565 / 1000 correct (56.50)

Iteration 700, loss = 1.1923
Checking accuracy on validation set
Got 565 / 1000 correct (56.50)



# Parte III. Desafío abierto de CIFAR-10

En esta sección, podés experimentar con cualquier arquitectura CNN que desee en CIFAR-10.

Ahora es tu trabajo experimentar con arquitecturas, hiperparámetros, funciones de pérdida y optimizadores para entrenar un modelo que logre ** al menos un 70% ** de accuracy en el conjunto de validación de CIFAR-10 en 10 épocas. Podés utilizar las funciones check_accuracy y train que ya definimos más arriba. Podés utilizar la API `nn.Module` o` nn.Sequential`.

Describí lo que hiciste al final de este notebook.

Acá está la documentación oficial de la API para cada componente. 

* Capas en el paquete torch.nn: http://pytorch.org/docs/stable/nn.html
* Activaciones: http://pytorch.org/docs/stable/nn.html#non-linear-activations
* Funciones de pérdida: http://pytorch.org/docs/stable/nn.html#loss-functions
* Optimizadores: http://pytorch.org/docs/stable/optim.html


### Cosas que podrías probar:
- ** Tamaño de kernel **: arriba usamos 5x5; ¿Serían más eficientes los kernels más pequeños?
- ** Número de kernels **: arriba usamos 32 kernels. ¿Más o menos lo hacen mejor?
- ** Pooling vs Strided Convolution **: ¿Utilizas el pooling máximo o simplemente convoluciones con stride?
- ** Normalización por lotes **: intente agregar la normalización por lotes espacial después de las capas de convolución ¿Tus redes se entrenan más rápido?
- ** Arquitectura de red **: la red anterior tiene dos capas de parámetros entrenables. ¿Puede hacerlo mejor con una red profunda? Las buenas arquitecturas para probar incluyen:
    - [conv-relu-pool]xN -> [afín]xM -> [softmax o SVM]
    - [conv-relu-conv-relu-pool]xN -> [affine]xM -> [softmax o SVM]
    - [batchnorm-relu-conv]xN -> [affine]xM -> [softmax o SVM]
- ** Pooling promedio global **: en lugar de aplanar y luego tener varias capas afines, realice convoluciones hasta que su imagen se vuelva pequeña (7x7 más o menos) y luego realice una operación de pooling promedio para obtener una imagen de imagen 1x1 (1, 1, Filter #), que luego se transforma en un vector (Filter #). Esto se utiliza en la [Red Inception de Google] (https://arxiv.org/abs/1512.00567) (consulte la Tabla 1 para conocer su arquitectura).
- ** Regularización **: agregue regularización de pesos, o tal vez Dropout.

### Consejos para el entrenamiento
Para cada arquitectura de red que pruebe, debe ajustar la tasa de aprendizaje y otros hiperparámetros. Al hacer esto, hay un par de cosas importantes a tener en cuenta:

- Si los hiperparámetros funcionan bien, deberías ver una mejora en unos pocos cientos de iteraciones
- Recordá el enfoque de grueso a fino para el ajuste de hiperparámetros: comenzá probando una amplia gama de hiperparámetros para solo unas pocas iteraciones de entrenamiento para encontrar las combinaciones de parámetros que funcionan.
- Una vez que hayas encontrado algunos conjuntos de hiperparámetros que parecen funcionar, buscá con más precisión alrededor de estos hiperparámetros. Puede que tengas que entrenar para más épocas.
- Tenés que utilizar el conjunto de validación para la búsqueda de hiperparámetros y guardarte el conjunto de prueba para evaluar tu arquitectura con los mejores parámetros seleccionados por el conjunto de validación.

### Ir más allá
Si te sentís aventurero, hay muchas otras features que podés implementar para intentar mejorar tu rendimiento. ** No es necesario ** que implementes ninguno de estos, ¡pero no te perdás la diversión si tenés tiempo!

- Optimizadores alternativos: podés probar Adam, Adagrad, RMSprop, etc.
- Funciones de activación alternativas como Leaky ReLU, PReLU, ELU o MaxOut.
- Aumento de datos
- Nuevas arquitecturas
  - [ResNets] (https://arxiv.org/abs/1512.03385) donde la entrada de la capa anterior se agrega a la salida.
  - [DenseNets] (https://arxiv.org/abs/1608.06993) donde las entradas de las capas anteriores se concatenan juntas.
  - [Este blog tiene una descripción detallada] (https://chatbotslife.com/resnets-highwaynets-and-densenets-oh-my-9bb15918ee32)

### ¡Divertite y feliz entrenamiento!

In [15]:
################################################################################
# TODO:                                                                        #         
# Experimentá con arquitecturas, optimizadores e hiperparámetros.              #
# Logre AL MENOS 70% de accuracy en el conjunto de validación en 10 épocas.    #
#                                                                              #
# Tené en cuenta que podés usar la función check_accuracy para evaluar tanto   #
# el conjunto de prueba como el conjunto de validación, pasando loader_test o  #
# loader_val como segundo argumento para check_accuracy. No deberías tocar el  #
# conjunto de prueba hasta que hayas elegido tu arquitectura e hiperparámetros.#
# Solo usás el conjunto de prueba una vez para informar un puntaje al final    #
################################################################################
model = None
optimizer = None
#############################################################################
# ***** INICIO DE TU CÓDIGO (NO BORRES NI MODIFIQUES ESTA LÍNEA) ***** 


class Modelito(nn.Module):

    def __init__(self, in_channel, channel_1, channel_2, channel_3, num_classes):
        super().__init__()

        self.capa1 = nn.Conv2d(in_channel, channel_1, kernel_size = 3, padding = 1)
        nn.init.kaiming_normal_(self.capa1.weight)
        self.capa2 = nn.Conv2d(channel_1, channel_2, kernel_size = 3, padding = 1)
        nn.init.kaiming_normal_(self.capa2.weight)
        self.capa3 = nn.Conv2d(channel_2, channel_3, kernel_size = 3, padding = 1)
        nn.init.kaiming_normal_(self.capa3.weight)
        self.salida = nn.Linear(16 * 4* 4, num_classes)
        nn.init.kaiming_normal_(self.salida.weight)
        self.dropout1 = nn.Dropout2d(0.2)
        self.flatten =  nn.Flatten()
        self.batchnorm1 = nn.BatchNorm2d(channel_1)
        self.batchnorm2 = nn.BatchNorm2d(channel_2)
        self.batchnorm3 = nn.BatchNorm2d(channel_3)


    def forward(self, x):
        
        x = F.relu(self.batchnorm1(self.capa1(x)))

        
        x = F.max_pool2d( x ,kernel_size = 3, padding =1)

        x = self.dropout1(x)
        
        x = F.relu(self.batchnorm2(self.capa2(x)))

        x = F.max_pool2d(x ,kernel_size = 3, padding =1)

        x = self.dropout1(x)
        
        x = F.relu(self.batchnorm3(self.capa3(x)))
       
        
        x =  self.salida(self.flatten(x))


        return x

in_channel = 3
channel_1 = 64 
channel_2 = 32
channel_3 = 16
num_classes = 10
learning_rate = 1e-2

model = Modelito(in_channel, channel_1 , channel_2 , channel_3 , num_classes )
optimizer = optim.SGD(model.parameters(), lr=learning_rate, momentum=0.9, nesterov=True)
# ***** FINAL DE TU CÓDIGO (NO BORRES NI MODIFIQUES ESTA LÍNEA) *****       #
#############################################################################
#                          FINAL DE TU CÓDIGO                               #       
#############################################################################

# Tenés que llegar a un 70% de accuracy
train_part34(model, optimizer, epochs=10)

Epoch 0
Iteration 0, loss = 2.4675
Checking accuracy on validation set
Got 128 / 1000 correct (12.80)

Iteration 100, loss = 1.6889
Checking accuracy on validation set
Got 426 / 1000 correct (42.60)

Iteration 200, loss = 1.6153
Checking accuracy on validation set
Got 494 / 1000 correct (49.40)

Iteration 300, loss = 1.7208
Checking accuracy on validation set
Got 493 / 1000 correct (49.30)

Iteration 400, loss = 1.6439
Checking accuracy on validation set
Got 524 / 1000 correct (52.40)

Iteration 500, loss = 1.6531
Checking accuracy on validation set
Got 542 / 1000 correct (54.20)

Iteration 600, loss = 1.3639
Checking accuracy on validation set
Got 548 / 1000 correct (54.80)

Iteration 700, loss = 1.2675
Checking accuracy on validation set
Got 568 / 1000 correct (56.80)

Epoch 1
Iteration 0, loss = 1.1402
Checking accuracy on validation set
Got 549 / 1000 correct (54.90)

Iteration 100, loss = 1.3347
Checking accuracy on validation set
Got 589 / 1000 correct (58.90)

Iteration 200, los

## Describí lo que hiciste

En la celda a continuación, tenés que escribir una explicación de lo que hiciste, las funciones adicionales que implementaste y / o los gráficos que realizaste en el proceso de entrenamiento y evaluación de tu red.

- En primer lugar decidí construir un modelo basándome en las sugerencias propuestas. El mismo quedó conformado por  3 capas convolucionales y 2 de Max Pooling. A su vez, bajé el número del Kernel de 5x5 a 3x3.

- Agregué Batch Normalization después de cada convolución

- La estructura quedó conformada como:

CV1 - BN - Relu - Max pooling

CV2 - BN - Relu - Max Pooling 

CV3 - BN - Relu

-Las capas convolucionales tuvieron de salida 64 canales, 32 canales y 16 canales. Esto implicó agregar más capas a los modelos anteriores.

- Agregué Dropout dos veces, luego de realizar los Max pooling

-Jugué con la cantidad de dropout y el Learning Rate, siendo las medidas óptimas : 
learning_rate = 0.01
Dropoout = 0.2

---



## Conjunto de prueba: ejecute esto solo una vez

Ahora que obtuvimos un resultado con el que estamos contentos, probamos nuestro modelo final en el conjunto de prueba (que debeés almacenar en best_model). Pensá en cómo se compara esto con la precisión de tu conjunto de validación.

In [16]:
best_model = model
check_accuracy_part34(loader_test, best_model)

Checking accuracy on test set
Got 7006 / 10000 correct (70.06)


Notamos que la precisión en el conjunto de Test casi que no varía en relación a la precisión en el conjunto de Validación. Creemos que esto se debe en gran medida a que el número total de iteraciones en el conjunto de validacióón es bastante parecido al núero total de iteraciones en el conjunto de Test.